 # Locate Sources with Array of Arrays
<img src="pictures/Grenzgletscher25_oben.jpg" alt="Grenzgletscher 25" width="600"/> <img src="pictures/Search_2025.jpg" alt="Grenzgletscher 25" width="500"/>

### Back Azimuth Estimation Using fk-Analysis 
 by J. Wassermann 





In [ ]:
# Import 
%matplotlib inline 

#import matplotlib
from obspy import *
from obspy.core import AttribDict
from obspy.core.inventory.inventory import Inventory
from obspy.clients.filesystem import sds
from obspy.clients.fdsn import Client
import obspy_arraytools as AA
import obspy.signal.util as util
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.colorbar import ColorbarBase
from matplotlib.colors import Normalize
import matplotlib.colors as colors
import matplotlib.cm as cm
import numpy as np
import csv

SDS_DATA_PATH = "/Users/jowa/Skience2026_Data/data_sds"

In [ ]:
def run(par):
    client = sds.Client(sds_root = par.sds_root)
    fp=open(par.phasefile,"r")
    while 1:
        line = fp.readline()
        if not line: break
        data = line.split(',')
        #we add tt_min and tt_max to be sure that the data frame is large enough
        if data[1] == "short_signal\n" or data[1] == "icequake\n":
            start = UTCDateTime(data[0])-par.pre_trigger      
            end = start + par.w_mig #UTCDateTime(data[1]) 
        try:
            for arraystats in par.arrays:
                sz = Stream()
                inv= Inventory()
                i = 0
                #reading in data and metadata of the array(s)
                for station in arraystats:
                    net,stat,loc,chan=station.split('.')
                    try:
                        tr = client.get_waveforms(network=net,station=stat,location=loc,channel=chan, starttime=start, endtime=end)
                        ii = read_inventory("./stationxml/station_%s_%s.xml"%(net,stat))
                        tr.merge()
                        if tr[0].stats.endtime < end or tr[0].stats.starttime > start:
                            print("trace ",tr, "too short")
                        else: 
                            sz += tr
                            inv += ii
                    except:
                        print("%s not loaded ...",station)
                        pass
                
                sz.merge()
                print(sz)
                sz.detrend("linear")
                sz.attach_response(inv)
        
                #we restrict ourself to the vertical components.... at least for now
                vc = sz.select(component="Z")
        
                #veeeery convenient function 
                # handles all geometry issues of the array
                grenz = AA.SeismicArray("",inv)
                grenz.inventory_cull(vc)
        
                print("Center of gravity: ",grenz.center_of_gravity)
                outray = 0.
                #we use here a covariance based FK-analysis
                outray = grenz.fk_analysis(vc, frqlow=par.fl, frqhigh=par.fh, prefilter=True,\
                         static3d=False, array_response=False,vel_corr=4.8, wlen=par.win_len,\
                         wfrac=par.win_frac,sec_km=True,
                         slx=(par.sll_x,par.slm_x),sly=(par.sll_y,par.slm_y),
                         sls=par.sl_s)      
            
                res = [outray.win_starttimes,outray.max_rel_power,outray.max_abs_power,outray.max_pow_baz,outray.max_pow_slow]
                res = np.asarray(res)
                net,stat,_,_ = arraystats[0].split(".")
                # save the output as csv file
                outfile = "%s/ARY-%s_%s-%s.csv"%(par.output_path,start,net,stat[:-2])
                with open(outfile, 'w', newline='') as csvfile:
                    writer = csv.writer(csvfile, dialect='unix')
                    writer.writerows(res.T)

    #############################################
    # Plotting .....
    #############################################

                hist_baz, bins_baz = np.histogram(outray.max_pow_baz, bins=int(360/2), \
                                    range = (0,360), weights = outray.max_rel_power, density = False)
                max_baz =  np.argmax(hist_baz)
                print(bins_baz[max_baz])

                xlocator = mdates.AutoDateLocator()

                fig = plt.figure()
                axbaz = fig.add_subplot(212)
                axtrace = fig.add_subplot(211,sharex=axbaz)
           
                axtrace.plot(vc[0].times("matplotlib"),vc[0].data,'k')
                axtrace.ticklabel_format(axis='y', style='sci', scilimits=(-2,2))
                axtrace.tick_params( axis='x',labelbottom='off')
                axtrace.set_ylabel('[a.u.]',color='k')
                for tl in axtrace.get_yticklabels():
                    tl.set_color('k')
            
                npts = vc[0].stats.npts
                nsamp = int(par.win_len*vc[0].stats.sampling_rate)
                axtrace.axvline(vc[0].times("matplotlib")[int(npts/6)], ls = "--",color="g")
                axtrace.axvline(vc[0].times("matplotlib")[int(npts/6)+nsamp], ls = "--",color="g")
                timeb = [number.matplotlib_date for number in outray.win_starttimes]
                axbaz.scatter(timeb, outray.max_pow_baz,c=outray.max_rel_power,cmap='Reds',s=15)
                axbaz.set_ylabel('[deg]')
                axbaz.set_ylim(0,360)
            
                axbaz.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
                axbaz.xaxis.set_major_locator(xlocator)

                fig.autofmt_xdate()
                
                plt.savefig("%s/ARRAY_%s-%s_%s.png"%(par.figure_path,net,stat[:-2],start))
                #plt.show()
        
                phi = outray.max_pow_baz*2*np.pi/360
                hist_baz, bins_baz = np.histogram(phi, bins=int(360/2),
                                                            range = (0,2*np.pi),
                                                            weights = outray.max_rel_power,
                                                            density = True)

                cmap = cm.viridis
                bins_number = 180  # the [0, 360) interval will be subdivided into this
                            # number of equal bins
                bins = np.linspace(0.0, 2 * np.pi, bins_number + 1)
                angles = 2 * np.pi * phi/360.

                width = 2 * np.pi / bins_number
                ax = plt.subplot(1, 1, 1, projection='polar')
                ax.set_theta_zero_location("N")
                ax.set_theta_direction(-1)
                ax.set_yticklabels([])
            
                bars = ax.bar(bins_baz[:-1], hist_baz, width=width,bottom=0.0,alpha=.7,color="orange")
                plt.savefig("%s/ROSE_%s-%s_%s.png"%(par.figure_path,net,stat[:-2],start),dpi = 72)
               # plt.show()
                plt.close("all")
        except:
            print("Didn't work \n")
            pass
            
                                                                                                                                                     

In [ ]:
par = AttribDict()
par.arrays = [["4D.A1P1.SW.GPZ","4D.A1P2.SW.GPZ","4D.A1P3.SW.GPZ","4D.A1P4.SW.GPZ",\
              "4D.A1P5.SW.GPZ","4D.A1P6.SW.GPZ","4D.A1P7.SW.GPZ","4D.A1P8.SW.GPZ"],\
            ["4D.A2P1.SW.GPZ","4D.A2P2.SW.GPZ","4D.A2P3.SW.GPZ","4D.A2P4.SW.GPZ",\
              "4D.A2P5.SW.GPZ","4D.A2P6.SW.GPZ","4D.A2P7.SW.GPZ","4D.A2P8.SW.GPZ"],\
            ["4D.A3P1.SW.GPZ","4D.A3P2.SW.GPZ","4D.A3P3.SW.GPZ","4D.A3P4.SW.GPZ",\
              "4D.A3P5.SW.GPZ","4D.A3P6.SW.GPZ","4D.A3P7.SW.GPZ","4D.A3P8.SW.GPZ"]]
print("No of arrays to be used: ",len(par.arrays))
par.no_arrays = len(par.arrays)

par.output_path = "./Grenzgletscher_fk"
par.figure_path = "./Grenzgletscher_fk/figure"

par.fl=5.0 #lower frequency 
par.fh=80.00 #upper frequency limit
par.win_len=.2  #windowlenth of sliding window in seconds
par.w_mig = 4.
par.win_frac=0.1 #step width of sliding window
par.sll_x=-0.5 #lower bound of slowness s/km
par.slm_x=0.5 #upper bound of slowness s/km
par.sll_y=-0.5 #ditto but y axis
par.slm_y=0.5 #ditto
par.sl_s=0.025 #grid with for search area
par.thres_rel = 0.0 #semblance value for trigger
par.thres_slow = 0.6
par.pre_trigger = 0.1
par.sds_root = SDS_DATA_PATH
par.phasefile = "./RF_Richels/afewones.csv"


In [ ]:
run(par)


## More advanced plotting comes now ...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from obspy import read_inventory, UTCDateTime
import cartopy.crs as ccrs
import csv

from math import asin, atan2, cos, degrees, radians, sin

In [ ]:
# helper function for qiver
def get_point_at_distance(lat1, lon1, d, bearing, R=6371):
    """
    lat: initial latitude, in degrees
    lon: initial longitude, in degrees
    d: target distance from initial
    bearing: (true) heading in degrees
    R: optional radius of sphere, defaults to mean radius of earth

    Returns new lat/lon coordinate {d}km from initial, in degrees
    """
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    a = radians(bearing)
    lat2 = asin(sin(lat1) * cos(d/R) + cos(lat1) * sin(d/R) * cos(a))
    lon2 = lon1 + atan2(
        sin(a) * sin(d/R) * cos(lat1),
        cos(d/R) - sin(lat1) * sin(lat2)
    )
    return (degrees(lat2), degrees(lon2))

In [ ]:
# read in topo data which was download via the Swiss topo
#downloader of QGis and than exported to a shape file -
#we load the stuff into cartopy
import os
import cartopy.crs as ccrs
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature

shapefile = "./shape/grenz.shp"

shape_feature = ShapelyFeature(Reader(shapefile).geometries(),\
                               ccrs.PlateCarree(), facecolor='none')
#choosing a valid projection
proj = ccrs.TransverseMercator(
   central_latitude=45.9,
   central_longitude=7.8)

#restrict the plotting area
min_lon = 7.8148
max_lon = 7.87217
min_lat = 45.9197
max_lat = 45.9397


directory = os.fsencode(par.output_path)
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".csv"): 
        filetemp = filename[:-7]
        csv_input= "%s/%s"%(par.output_path, filetemp)

        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(projection=proj)
        ax.add_feature(shape_feature)
        ax.set_extent([min_lon,max_lon,min_lat,max_lat])


        stations_f = []
        inv = read_inventory("./stationxml/station_4D_A?P1.xml")
        for stat in inv:
            stat_id =  stat.get_contents()['channels'][0]
            coo = inv.get_coordinates(stat_id)
            ll = [stat[0].code, coo['longitude'], coo['latitude'], coo['elevation']]
            stations_f.append(ll)

        for name, lon, lat, elev in stations_f:
            ax.plot(lon,lat, "rv", markersize=5, transform=ccrs.PlateCarree())
            try:
                csvf = "%s%s.csv"%(csv_input,name[:-2])
                baz = []
                mcorr = []
                slow = []
                with open(csvf, 'r') as f:
                    reader = csv.reader(f)
                    for row in reader:
                        baz.append(float(row[3]))
                        mcorr.append(float(row[1]))
                        slow.append(float(row[-1]))

                baz = np.asarray(baz)
                mcorr = np.asarray(mcorr)
                slow = np.asarray(slow)
                for i in range(len(baz)):
                    if np.abs(mcorr[i]) > 0.7 and slow[i]<0.33:
                        y,x = get_point_at_distance(lat, lon, 1, baz[i])
                        dy = (y - lat)*10000
                        dx = (x - lon)*10000
                        dx = [lon,x]
                        dy = [lat,y]
                        ax.plot(dx,dy,"g-",alpha=np.abs(mcorr[i])/10,transform=ccrs.PlateCarree())
            except:
                continue

        fig.savefig("%s/%s.png"%(par.figure_path,filetemp), dpi=300)
        plt.show()
                                                                                